### 전체 mac 모수 확인

In [0]:
%sql 
WITH final_log as (
    SELECT device_id, log_create_time, id, label, name
    FROM   (
        SELECT
            device_id
            , log_create_time
            , rank() over (partition by device_id ORDER BY log_create_time desc) as rn
            , id 
            , label
            , name
        FROM eic_data_ods.tlamp_view.view__eim__nl_eim_inputlist
        WHERE 1=1
            AND platform_version != 'webOSTV 6.0'
            AND country_code = 'NG'
            AND date_ym >= '2024-01'
            AND date_ym <  '2025-12'
    ) 
    WHERE rn = 1
), mart_mac (
    select distinct device_id
    from   final_log
)
select count(1) from mart_mac 

### 모수에 대한 label_#, name_# 구하기 (# : 1,2,3,4)

In [0]:
# 나이지리아 TV에 연결된 모든 기기 모델명과 브랜드 확인
# * TV 1대당 최종 상태만 확인하면 됩니다

df = spark.sql(f'''
    WITH final_log as (
        SELECT device_id, log_create_time, id, label, name
        FROM   (
            SELECT
                device_id
                , log_create_time
                , rank() over (partition by device_id ORDER BY log_create_time desc) as rn
                , id 
                , label
                , name
            FROM eic_data_ods.tlamp_view.view__eim__nl_eim_inputlist
            WHERE 1=1
                AND platform_version != 'webOSTV 6.0'
                AND country_code = 'NG'
                AND date_ym >= '2024-01'
                AND date_ym <  '2025-12'
        ) 
        WHERE rn = 1
    ), mart_mac (
        select distinct device_id
        from   final_log
    ), mart_1 as (
        select device_id, first(label) as label_1, first(name) as name_1 -- 여러개가 찍히는 경우 첫번째행 출력
        from   final_log raw
        where  1=1 
          AND  id = 'HDMI_1'
          AND  exists (
            select 1 
            from final_log 
            where final_log.device_id = raw.device_id
             and final_log.log_create_time = raw.log_create_time
        )
        group by device_id
    ), mart_2 as (
        select device_id, first(label) as label_2, first(name) as name_2 -- 여러개가 찍히는 경우 첫번째행 출력
        from   final_log raw
        where  1=1 
          AND  id = 'HDMI_2'
          AND  exists (
            select 1 
            from final_log 
            where final_log.device_id = raw.device_id
             and final_log.log_create_time = raw.log_create_time
        )
        group by device_id
    ), mart_3 as (
        select device_id, first(label) as label_3, first(name) as name_3 -- 여러개가 찍히는 경우 첫번째행 출력
        from   final_log raw
        where  1=1 
          AND  id = 'HDMI_3'
          AND  exists (
            select 1 
            from final_log
            where final_log.device_id = raw.device_id
              and final_log.log_create_time = raw.log_create_time
        )
        group by device_id
    ), mart_4 as (
        select device_id, first(label) as label_4, first(name) as name_4 -- 여러개가 찍히는 경우 첫번째행 출력
        from   final_log raw
        where  1=1 
          AND  id = 'HDMI_4'
          AND  exists (
            select 1 
            from final_log 
            where final_log.device_id = raw.device_id
              and final_log.log_create_time = raw.log_create_time
        )
        group by device_id 
    ), mart_result as (
        select distinct device_id, label_1, name_1, label_2, name_2, label_3, name_3, label_4, name_4
        from   mart_mac
        left join mart_1 using(device_id)
        left join mart_2 using(device_id)
        left join mart_3 using(device_id)
        left join mart_4 using(device_id)
    )

    select row_number() over(order by device_id) as device_num   -- count(1) : 81,389
           , label_1, name_1, label_2, name_2, label_3, name_3, label_4, name_4
    from   mart_result
''')

df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save('s3://s3-lge-he-inbound-eic-dev/HEDS/1169')

In [0]:
display(df.count()) # 전체 모수와 같은지 확인

In [0]:
%sql 

WITH final_log as (
    SELECT distinct device_id, log_create_time, id, label, name
    FROM   (
        SELECT
            device_id
            , log_create_time
            , rank() over(partition by device_id order by log_create_time desc) as rn
            , id 
            , label
            , name
        FROM eic_data_ods.tlamp_view.view__eim__nl_eim_inputlist
        WHERE 1=1
            AND platform_version != 'webOSTV 6.0'
            AND country_code = 'NG'
            AND date_ym >= '2024-01'
            AND date_ym <  '2025-12'
    ) 
    WHERE rn = 1
)

select *
from   final_log
where  device_id = '4eac7dffb6a15b7c6da1ff4c92395a2408ba06dcb141be1592a8d59396d674c7'